In [1]:
import os
import sys
import fire
import torch
import transformers
from transformers import Trainer, AutoModel
from datasets import load_dataset
from datasets import load_from_disk

from peft import (
    prepare_model_for_int8_training,
)
from src.mola_mapping_hacked import get_peft_model
from src.mola_lora_hacked import LoraConfig
from src.mola_peft_model_hacked import set_peft_model_state_dict_moe

from transformers import LlamaTokenizer, AutoConfig
from transformers import AutoTokenizer
from src.mola_modeling_mistral_hacked import MistralForCausalLM_d
from src.mistralconfig import MistralConfig
from utils.prompter import Prompter
from transformers import TrainerCallback


os.environ["WANDB_DISABLED"] = "true"

import random

seed = 10
random.seed(seed)
torch.manual_seed(0)

/home/haskari/miniconda3/envs/alphalora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'prepare_model_for_int8_training' from 'peft' (/home/haskari/miniconda3/envs/alphalora/lib/python3.10/site-packages/peft/__init__.py)

In [3]:
from ast import literal_eval
check='[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]'
my_list=literal_eval(check)

print(len(my_list))
sum(my_list)


28


56

In [26]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

In [27]:
model = AutoModel.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.34s/it]


In [1]:
import json

In [ ]:
with open('datasets/glue_mrpc_train.json', 'r') as f:
    mrpc_train_json=json.load(f)

In [ ]:
len(mrpc_train_json)

In [ ]:
len(mrpc_test_json)

In [17]:
from datasets import load_dataset, load_from_disk

In [2]:
mrpc=load_from_disk('datasets/glue_mrpc_all.hf')

In [3]:
mrpc['test'][0]

{'input': '',
 'instruction': "Tell me if the statements equivalent, not equivalent.\nSentence 1: PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .\nSentence 2: Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .\n",
 'output': 'Answer: equivalent.',
 'answer': 'equivalent'}

In [17]:
rte=load_from_disk('datasets/glue_rte_all.hf')

In [20]:
rte['train'][2]

{'input': '',
 'instruction': 'Tell me if the statements entailment, not entailment.\nSentence 1: Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.\nSentence 2: Herceptin can be used to treat breast cancer.\n',
 'output': 'Answer: entailment.',
 'answer': 'entailment'}

In [19]:
rte['test'][2]

{'input': '',
 'instruction': 'Tell me if the statements entailment, not entailment.\nSentence 1: Cairo is now home to some 15 million people - a burgeoning population that produces approximately 10,000 tonnes of rubbish per day, putting an enormous strain on public services. In the past 10 years, the government has tried hard to encourage private investment in the refuse sector, but some estimate 4,000 tonnes of waste is left behind every day, festering in the heat as it waits for someone to clear it up. It is often the people in the poorest neighbourhoods that are worst affected. But in some areas they are fighting back. In Shubra, one of the northern districts of the city, the residents have taken to the streets armed with dustpans and brushes to clean up public areas which have been used as public dumps.\nSentence 2: 15 million tonnes of rubbish are produced daily in Cairo.\n',
 'output': 'Answer: not entailment.',
 'answer': 'not entailment'}

In [9]:
import json

with open('datasets/glue_rte_test.json', 'r') as f:
    rte_test_json=json.load(f)

In [15]:
rte_test_json[2]

{'input': '',
 'instruction': 'Tell me if the statements entailment, not entailment.\nSentence 1: Cairo is now home to some 15 million people - a burgeoning population that produces approximately 10,000 tonnes of rubbish per day, putting an enormous strain on public services. In the past 10 years, the government has tried hard to encourage private investment in the refuse sector, but some estimate 4,000 tonnes of waste is left behind every day, festering in the heat as it waits for someone to clear it up. It is often the people in the poorest neighbourhoods that are worst affected. But in some areas they are fighting back. In Shubra, one of the northern districts of the city, the residents have taken to the streets armed with dustpans and brushes to clean up public areas which have been used as public dumps.\nSentence 2: 15 million tonnes of rubbish are produced daily in Cairo.\n',
 'output': 'Answer: not entailment.',
 'answer': 'not entailment'}